In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, StandardScaler,PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pickle
from scipy.stats import mannwhitneyu

In [8]:
df_flights = pd.read_csv(f'./data/data_usa/flights_final.csv',low_memory=False)

In [9]:
df_flights['DATE'] = pd.to_datetime(df_flights['DATE'])

In [10]:
df_flights['SCHEDULED_DEPARTURE'] = pd.to_datetime(df_flights['SCHEDULED_DEPARTURE']).dt.time
df_flights['SCHEDULED_ARRIVAL'] = pd.to_datetime(df_flights['SCHEDULED_ARRIVAL']).dt.time

C:\Users\Melissa\AppData\Local\Temp\ipykernel_5704\4276420066.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_flights['SCHEDULED_DEPARTURE'] = pd.to_datetime(df_flights['SCHEDULED_DEPARTURE']).dt.time
C:\Users\Melissa\AppData\Local\Temp\ipykernel_5704\4276420066.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_flights['SCHEDULED_ARRIVAL'] = pd.to_datetime(df_flights['SCHEDULED_ARRIVAL']).dt.time


#### Séparer le dataset en train (3 premières semaines de janvier) et test (dernière semaine de janvier)

In [11]:
df_train = df_flights[df_flights['DATE'].apply(lambda x:x.date()) < datetime.date(2015, 1, 23)]
df_test  = df_flights[df_flights['DATE'].apply(lambda x:x.date()) > datetime.date(2015, 1, 23)]

In [12]:
df_train.head()

,DATE,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY
0,2015-01-01,AS,ANC,SEA,00:05:00,23:54:00,-11.0,205.0,194.0,2331,04:30:00,04:08:00,-22.0
1,2015-01-01,AA,LAX,PBI,00:10:00,00:02:00,-8.0,280.0,279.0,3750,07:50:00,07:41:00,-9.0
2,2015-01-01,US,SFO,CLT,00:20:00,00:18:00,-2.0,286.0,293.0,3696,08:06:00,08:11:00,5.0
3,2015-01-01,AA,LAX,MIA,00:20:00,00:15:00,-5.0,285.0,281.0,3770,08:05:00,07:56:00,-9.0
4,2015-01-01,AS,SEA,ANC,00:25:00,00:24:00,-1.0,235.0,215.0,2331,03:20:00,02:59:00,-21.0


In [13]:
def create_df(df, carrier):
    df2 = df[df['AIRLINE'] == carrier][['DATE','SCHEDULED_DEPARTURE','SCHEDULED_ARRIVAL',
                                    'ORIGIN_AIRPORT','DESTINATION_AIRPORT','DEPARTURE_DELAY']]
    df2.dropna(how = 'any', inplace = True)
    df2['weekday'] = df2['DATE'].apply(lambda x:x.weekday())
    #____________________
    # delete delays > 1h
    df2['DEPARTURE_DELAY'] = df2['DEPARTURE_DELAY'].apply(lambda x:x if x < 60 else np.nan)
    df2.dropna(how = 'any', inplace = True)
    #_________________
    # formating times
    fct = lambda x:x.hour*3600+x.minute*60+x.second
    df2['heure_depart'] = df2['SCHEDULED_DEPARTURE'].apply(fct)
    df2['heure_arrivee'] = df2['SCHEDULED_ARRIVAL'].apply(fct)
    df3 = df2.groupby(['heure_depart', 'heure_arrivee', 'ORIGIN_AIRPORT','weekday',]).agg({
        'DEPARTURE_DELAY': 'mean'  # Calculez la moyenne seulement pour les retards de départ
    }).reset_index()
    return df3

In [18]:
carrier = 'AA'
df_model = create_df(df_train, carrier)

In [19]:
df_model

,heure_depart,heure_arrivee,ORIGIN_AIRPORT,weekday,DEPARTURE_DELAY
0,300,17640,LAX,0,-6.500000
1,300,17640,LAX,1,-0.333333
2,300,17640,LAX,2,2.500000
3,300,17640,LAX,3,-1.666667
4,300,17640,LAX,4,9.000000
...,...,...,...,...,...
15779,86100,30360,SEA,2,-2.000000
15780,86100,30360,SEA,3,2.666667
15781,86100,30360,SEA,4,-2.500000
15782,86100,30360,SEA,5,-1.500000


In [20]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df_model['ORIGIN_AIRPORT'])
#_________________________________________________________
zipped = zip(integer_encoded, df_model['ORIGIN_AIRPORT'])
label_airports = list(set(list(zipped)))
label_airports.sort(key = lambda x:x[0])
#_________________________________________________
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
#_________________________________________________
b = np.array(df_model[['heure_depart', 'heure_arrivee']])
X = np.hstack((onehot_encoded, b))
Y = np.array(df_model['DEPARTURE_DELAY'])
Y = Y.reshape(len(Y), 1)

In [68]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [69]:
del df_model

# Entraîner le modèle sur les trois premières semaines de janvier

## Ridge Regression

Bon point de départ pour la prédiction, facile à interpréter et nous sert de baselines pour comparer les performances des autres modèles

In [16]:
# Step 1: Setup the pipeline with PolynomialFeatures, StandardScaler, and Ridge
pipeline_ridge = Pipeline([
    ('poly', PolynomialFeatures()),
    ('scaler', StandardScaler()),  # Add a scaler to normalize the data
    ('ridge', Ridge())
])

# Step 2: Define the parameter grid
param_grid_ridge = {
    'poly__degree': [1, 2],  # Degrees of the polynomial to test
    'ridge__alpha': [i / 10.0 for i in range(0, 20, 2)]  # Alpha values to test
}

# Step 3: Initialize GridSearchCV
grid_search_ridge = GridSearchCV(pipeline_ridge, param_grid_ridge, cv=5, verbose=1)

# Assuming X_train and Y_train are defined
grid_search_ridge.fit(X_train, Y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('poly', PolynomialFeatures()),
                                       ('scaler', StandardScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'poly__degree': [1, 2],
                         'ridge__alpha': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2,
                                          1.4, 1.6, 1.8]},
             verbose=1)

In [17]:
# Step 6: Best model parameters and score
print("Best parameters found:", grid_search_ridge.best_params_)
print("Best cross-validation score (MSE):", grid_search_ridge.best_score_)

Best parameters found: {'poly__degree': 1, 'ridge__alpha': 1.8}
Best cross-validation score (MSE): 0.07832608619896408


In [18]:
# Step 7: Test set performance
best_model_ridge = grid_search_ridge.best_estimator_
predictions_ridge = best_model_ridge.predict(X_test)
test_mse_ridge = mean_squared_error(Y_test, predictions_ridge)
print("Test set MSE:", test_mse_ridge)
print('Ecart = {:.2f} min'.format(np.sqrt(test_mse_ridge)))

Test set MSE: 164.47061972935217
Ecart = 12.82 min


In [23]:
# save the iris classification model as a pickle file
model_pkl_file = f"./results/ridge_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(best_model_ridge, file)

## Lasso

In [25]:
# Configuration du pipeline avec Lasso
pipeline_lasso = Pipeline([
    ('poly', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    ('lasso', Lasso(max_iter=25000))
])

# Grille de paramètres pour Lasso
param_grid_lasso = {
    'poly__degree': [1, 2],
    'lasso__alpha': [0.001, 0.01, 0.1, 1, 10]  # Différents niveaux de régularisation
}

# Initialisation de GridSearchCV
grid_search_lasso = GridSearchCV(pipeline_lasso, param_grid_lasso, cv=5, verbose=1)
grid_search_lasso.fit(X_train, Y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\Melissa\anaconda3\envs\Sorbonne\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.033e+03, tolerance: 1.639e+02
  model = cd_fast.enet_coordinate_descent(
C:\Users\Melissa\anaconda3\envs\Sorbonne\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.762e+02, tolerance: 1.646e+02
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('poly', PolynomialFeatures()),
                                       ('scaler', StandardScaler()),
                                       ('lasso', Lasso(max_iter=25000))]),
             param_grid={'lasso__alpha': [0.001, 0.01, 0.1, 1, 10],
                         'poly__degree': [1, 2]},
             verbose=1)

In [26]:
# Affichage des meilleurs paramètres et score
print("Best parameters found:", grid_search_lasso.best_params_)
print("Best cross-validation score:", grid_search_lasso.best_score_)

Best parameters found: {'lasso__alpha': 0.1, 'poly__degree': 2}
Best cross-validation score: 0.08347925069283328


In [27]:
# Step 7: Test set performance
best_model_lasso = grid_search_lasso.best_estimator_
predictions_lasso = best_model_lasso.predict(X_test)
test_mse_lasso = mean_squared_error(Y_test, predictions_lasso)
print("Test set MSE:", test_mse_lasso)
print('Ecart = {:.2f} min'.format(np.sqrt(test_mse_lasso)))

Test set MSE: 163.88125479479848
Ecart = 12.80 min


In [28]:
# save the iris classification model as a pickle file
model_pkl_file = f"./results/lasso_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(best_model_lasso, file)

## Elastic Net

In [29]:
# Configuration du pipeline avec ElasticNet
pipeline_net = Pipeline([
    ('poly', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    ('elasticnet', ElasticNet(max_iter=20000))
])

# Grille de paramètres pour ElasticNet
param_grid_net = {
    'poly__degree': [1, 2],
    'elasticnet__alpha': [0.001, 0.01, 0.1, 1],
    'elasticnet__l1_ratio': [0.2, 0.5, 0.8]  # Proportion de la pénalité L1 dans le terme de régularisation
}

# Initialisation de GridSearchCV
grid_search_net = GridSearchCV(pipeline_net, param_grid_net, cv=5, verbose=1)
grid_search_net.fit(X_train, Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('poly', PolynomialFeatures()),
                                       ('scaler', StandardScaler()),
                                       ('elasticnet',
                                        ElasticNet(max_iter=20000))]),
             param_grid={'elasticnet__alpha': [0.001, 0.01, 0.1, 1],
                         'elasticnet__l1_ratio': [0.2, 0.5, 0.8],
                         'poly__degree': [1, 2]},
             verbose=1)

In [31]:
# Affichage des meilleurs paramètres et score
print("Best parameters found:", grid_search_net.best_params_)
print("Best cross-validation score:", grid_search_net.best_score_)

Best parameters found: {'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.8, 'poly__degree': 2}
Best cross-validation score: 0.08317874212130069


In [32]:
# Step 7: Test set performance
best_model_net = grid_search_net.best_estimator_
predictions_net = best_model_net.predict(X_test)
test_mse_net = mean_squared_error(Y_test, predictions_net)
print("Test set MSE:", test_mse_net)
print('Ecart = {:.2f} min'.format(np.sqrt(test_mse_net)))

Test set MSE: 163.97120666309988
Ecart = 12.81 min


In [33]:
# save the iris classification model as a pickle file
model_pkl_file = f"./results/net_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(best_model_net, file)

## RandomForest

In [34]:
# Configuration du pipeline sans PolynomialFeatures pour RandomForest
Y_train_forest = Y_train.squeeze() 
pipeline_forest = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor())
])

# Grille de paramètres pour RandomForest
param_grid_forest = {
    'rf__n_estimators': [100, 200, 300],  # Nombre d'arbres
    'rf__max_depth': [10, 20, 30]  # Profondeur maximale des arbres
}

# Initialisation de GridSearchCV
grid_search_forest = GridSearchCV(pipeline_forest, param_grid_forest, cv=5, verbose=1)
grid_search_forest.fit(X_train, Y_train_forest)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rf', RandomForestRegressor())]),
             param_grid={'rf__max_depth': [10, 20, 30],
                         'rf__n_estimators': [100, 200, 300]},
             verbose=1)

In [35]:
# Affichage des meilleurs paramètres et score
print("Best parameters found:", grid_search_forest.best_params_)
print("Best cross-validation score:", grid_search_forest.best_score_)

Best parameters found: {'rf__max_depth': 10, 'rf__n_estimators': 300}
Best cross-validation score: 0.09558551063083334


In [36]:
# Step 7: Test set performance
best_model_forest = grid_search_forest.best_estimator_
predictions_forest = best_model_forest.predict(X_test)
test_mse_forest = mean_squared_error(Y_test, predictions_forest)
print("Test set MSE:", test_mse_forest)
print('Ecart = {:.2f} min'.format(np.sqrt(test_mse_forest)))

Test set MSE: 162.16544050720663
Ecart = 12.73 min


In [37]:
# save the iris classification model as a pickle file
model_pkl_file = f"./results/forest_model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(best_model_forest, file)

### Tester les modèles sur la dernière semaine de janvier

In [15]:
carrier = "AA"
df_model_test = create_df(df_test, carrier)    
df_model_test

,heure_depart,heure_arrivee,ORIGIN_AIRPORT,weekday,DEPARTURE_DELAY
0,60,31740,LAX,6,-3.000000
1,300,5880,CLT,0,-1.000000
2,300,5880,CLT,2,-5.000000
3,300,5880,CLT,5,18.000000
4,300,5880,CLT,6,-9.000000
...,...,...,...,...,...
122667,86340,30900,SFO,2,-0.076923
122668,86340,30900,SFO,3,8.214286
122669,86340,30900,SFO,4,4.357143
122670,86340,30900,SFO,5,-0.785714


#### Transformer les codes des aéroports en matrice encodés one-hot + Merge avec d'autres colonnes numériques du dataframe

In [21]:
label_conversion = dict()
for s in label_airports:
    label_conversion[s[1]] = s[0]

df_model_test['ORIGIN_AIRPORT'].replace(label_conversion, inplace = True)

for index, label in label_airports:
    temp = df_model_test['ORIGIN_AIRPORT'] == index
    temp = temp.apply(lambda x:1.0 if x else 0.0)
    if index == 0:
        matrix = np.array(temp)
    else:
        matrix = np.vstack((matrix, temp))
matrix = matrix.T

b = np.array(df_model_test[['heure_depart', 'heure_arrivee']])
X_test = np.hstack((matrix, b))
Y_test = np.array(df_model_test['DEPARTURE_DELAY'])
Y_test = Y_test.reshape(len(Y_test), 1)

In [53]:
del df_model_test

In [5]:
ridge_model = pickle.load(open('results/ridge_model.pkl', 'rb'))
lasso_model = pickle.load(open('results/lasso_model.pkl', 'rb'))
net_model = pickle.load(open('results/net_model.pkl', 'rb'))
forest_model = pickle.load(open('results/forest_model.pkl', 'rb'))

## Ridge

In [23]:
# best_model_ridge = grid_search_ridge.best_estimator_
predictions_ridge = ridge_model.predict(X_test)
test_mse_ridge = mean_squared_error(Y_test, predictions_ridge)
print("Test set MSE:", test_mse_ridge)
print('Ecart = {:.2f} min'.format(np.sqrt(test_mse_ridge)))

Test set MSE: 68.9144186245605
Ecart = 8.30 min


In [24]:
icount = 0
for i, val in enumerate(Y_test):
    if abs(val-predictions_ridge[i]) > 15: icount += 1
print("ecarts > 15 minutes: {}%".format(round((icount / len(predictions_ridge))*100,3)))

ecarts > 15 minutes: 4.952%


## Lasso

In [26]:
# best_model_lasso = grid_search_lasso.best_estimator_
predictions_lasso = lasso_model.predict(X_test)
test_mse_lasso = mean_squared_error(Y_test, predictions_lasso)
print("Test set MSE:", test_mse_lasso)
print('Ecart = {:.2f} min'.format(np.sqrt(test_mse_lasso)))

Test set MSE: 66.30482919277203
Ecart = 8.14 min


In [27]:
icount = 0
for i, val in enumerate(Y_test):
    if abs(val-predictions_lasso[i]) > 15: icount += 1
print("ecarts > 15 minutes: {}%".format(round((icount / len(predictions_lasso))*100,3)))

ecarts > 15 minutes: 4.695%


## Elastic Net

In [28]:
# best_model_net = grid_search_net.best_estimator_
predictions_net = net_model.predict(X_test)
test_mse_net = mean_squared_error(Y_test, predictions_net)
print("Test set MSE:", test_mse_net)
print('Ecart = {:.2f} min'.format(np.sqrt(test_mse_net)))

Test set MSE: 66.45032264012953
Ecart = 8.15 min


In [29]:
icount = 0
for i, val in enumerate(Y_test):
    if abs(val-predictions_net[i]) > 15: icount += 1
print("ecarts > 15 minutes: {}%".format(round((icount / len(predictions_net))*100,3)))

ecarts > 15 minutes: 4.742%


## RandomForest

In [30]:
# best_model_forest = grid_search_forest.best_estimator_
predictions_forest = forest_model.predict(X_test)
test_mse_forest = mean_squared_error(Y_test, predictions_forest)
print("Test set MSE:", test_mse_forest)
print('Ecart = {:.2f} min'.format(np.sqrt(test_mse_forest)))

Test set MSE: 69.18367851024573
Ecart = 8.32 min


In [31]:
icount = 0
for i, val in enumerate(Y_test):
    if abs(val-predictions_forest[i]) > 15: icount += 1
print("ecarts > 15 minutes: {}%".format(round((icount / len(predictions_forest))*100,3)))

ecarts > 15 minutes: 5.591%


## Test de significativité

In [32]:
def test_mannnwhitneyu(metric_model1, metric_model2, tested="métriques"):
    # Effectuer le test de Mann-Whitney
    stat, p_value = mannwhitneyu(metric_model1, metric_model2)

    print("Statistique U:", stat, "Valeur p:", p_value)
    if p_value < 0.01:
        print(f"Les différences de {tested} sont statistiquement significatives.")
    else:
        print(f"Aucune différence significative de {tested} détectée.")

In [36]:
test_mannnwhitneyu(test_mse_ridge, test_mse_forest, tested="MSE")
test_mannnwhitneyu(test_mse_ridge, test_mse_lasso, tested="MSE")
test_mannnwhitneyu(test_mse_ridge, test_mse_net, tested="MSE")

Statistique U: 0.0 Valeur p: 1.0
Aucune différence significative de MSE détectée.
Statistique U: 1.0 Valeur p: 1.0
Aucune différence significative de MSE détectée.
Statistique U: 1.0 Valeur p: 1.0
Aucune différence significative de MSE détectée.


In [37]:
test_mannnwhitneyu(np.sqrt(test_mse_ridge), np.sqrt(test_mse_forest), tested="RMSE")
test_mannnwhitneyu(np.sqrt(test_mse_ridge), np.sqrt(test_mse_lasso), tested="RMSE")
test_mannnwhitneyu(np.sqrt(test_mse_ridge), np.sqrt(test_mse_net), tested="RMSE")

Statistique U: 0.0 Valeur p: 1.0
Aucune différence significative de RMSE détectée.
Statistique U: 1.0 Valeur p: 1.0
Aucune différence significative de RMSE détectée.
Statistique U: 1.0 Valeur p: 1.0
Aucune différence significative de RMSE détectée.


In [38]:
test_mannnwhitneyu(test_mse_lasso, test_mse_forest, tested="MSE")
test_mannnwhitneyu(test_mse_ridge, test_mse_lasso, tested="MSE")
test_mannnwhitneyu(test_mse_lasso, test_mse_net, tested="MSE")

Statistique U: 0.0 Valeur p: 1.0
Aucune différence significative de MSE détectée.
Statistique U: 1.0 Valeur p: 1.0
Aucune différence significative de MSE détectée.
Statistique U: 0.0 Valeur p: 1.0
Aucune différence significative de MSE détectée.


In [39]:
test_mannnwhitneyu(np.sqrt(test_mse_lasso), np.sqrt(test_mse_forest), tested="RMSE")
test_mannnwhitneyu(np.sqrt(test_mse_lasso), np.sqrt(test_mse_net), tested="RMSE")

Statistique U: 0.0 Valeur p: 1.0
Aucune différence significative de RMSE détectée.
Statistique U: 0.0 Valeur p: 1.0
Aucune différence significative de RMSE détectée.


In [40]:
test_mannnwhitneyu(test_mse_net, test_mse_forest, tested="MSE")

Statistique U: 0.0 Valeur p: 1.0
Aucune différence significative de MSE détectée.


In [41]:
test_mannnwhitneyu(np.sqrt(test_mse_net), np.sqrt(test_mse_forest), tested="RMSE")

Statistique U: 0.0 Valeur p: 1.0
Aucune différence significative de RMSE détectée.
